# Backtrader

En esta notebook realizamos una simulacion utilizando los modelos entrenados

In [1]:
! pip install backtrader;

In [8]:
import backtrader as bt
import pandas as pd

Importamos la data de test de los modelos, en todos los casos es la misma data excluyendo el target, consta del 2018 al 2020. Estos son datos que los modelos no han visto.

In [9]:
data = pd.read_csv('models/model_test_Forward_Return_1m.csv', index_col = 0, parse_dates=True)
data.head(3)

,Open,High,Low,Close,Volume,MACD,RSI,BB_High,BB_Mid,BB_Low,ATR,NATR,Currency_Volume,Adj Close,Month,Weekday,Ticker,Currency,Forward_Return_1m
Date,,,,,,,,,,,,,,,,,,,
2018-07-24,360.50,360.50,360.50,360.50,0.0,-1.705303e-13,0.000000,5.890262,5.890262,5.890262,3.526296e-25,9.781681e-26,0.000000e+00,358.496796,7,1,AMGN.BA,ARS,0.000000
2018-07-24,4.99,5.06,4.93,4.99,29389000.0,-3.568515e-02,54.042247,1.795840,1.753049,1.710258,1.408903e-01,2.823454e+00,1.466511e+08,4.766606,7,1,ABEV,USD,-0.118236
2018-07-24,5.64,5.74,5.62,5.69,546800.0,-1.256564e-01,49.939444,1.926558,1.881268,1.835979,1.809786e-01,3.180644e+00,3.111292e+06,5.248040,7,1,ELP,USD,-0.128295


In [10]:
data.index.is_monotonic, data.index[0], data.index[-1]

(True, Timestamp('2018-07-24 00:00:00'), Timestamp('2020-06-10 00:00:00'))

Importo el modelo

In [1]:
import joblib

In [15]:
model = joblib.load('models/trained_model_Forward_Return_1m.joblib')

C:\Users\carlm\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\carlm\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator ColumnTransformer from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\carlm\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator Pipeline from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [16]:
print(model)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('continuous', 'passthrough',
                                                  ['Open', 'High', 'Low',
                                                   'Close', 'Volume', 'MACD',
                                                   'RSI', 'BB_High', 'BB_Mid',
                                                   'BB_Low', 'ATR', 'NATR',
                                                   'Currency_Volume',
                                                   'Adj Close']),
                                                 ('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Month', 'Weekday', 'Ticker',
                                                   'Currency'])])),
                ('estimator',
                 XGBRegressor(base_score...
                              colsample_bytree=1, gamma=0, gpu_id=

In [17]:
print(model.steps[0]), print(model.steps[1])

('preprocessing', ColumnTransformer(transformers=[('continuous', 'passthrough',
                                 ['Open', 'High', 'Low', 'Close', 'Volume',
                                  'MACD', 'RSI', 'BB_High', 'BB_Mid', 'BB_Low',
                                  'ATR', 'NATR', 'Currency_Volume',
                                  'Adj Close']),
                                ('categorical',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['Month', 'Weekday', 'Ticker', 'Currency'])]))
('estimator', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=0,
   

(None, None)

In [23]:
data.iloc[-1]

Open                     1461.51
High                     1472.77
Low                      1454.36
Close                     1464.7
Volume                1.5881e+06
MACD                     34.3129
RSI                      67.7968
BB_High                  7.30012
BB_Mid                    7.2552
BB_Low                   7.21028
ATR                      31.9126
NATR                     2.17878
Currency_Volume      2.32609e+09
Adj Close                 1464.7
Month                          6
Weekday                        2
Ticker                     GOOGL
Currency                     USD
Forward_Return_1m      0.0355363
Name: 2020-06-10 00:00:00, dtype: object

In [24]:
model.predict(data)[-1]

C:\Users\carlm\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\compose\_column_transformer.py:440: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


0.043270826

Derivo un dataframe para usar en la simulacion que no contiene el target, ademas, convierto a variable numerica las variables de tipo string porque no son aceptadas por backtrader, podemos convertirlas de vuelta para mandarla al modelo.

In [26]:
from sklearn.preprocessing import LabelEncoder

simulation_data = data.drop('Forward_Return_1m', axis = 1)[:500]

ticker_encoder = LabelEncoder()
simulation_data['Ticker_Name'] = simulation_data['Ticker']
simulation_data['Ticker'] = ticker_encoder.fit_transform(simulation_data.Ticker)

currency_encoder = LabelEncoder()
simulation_data['Currency'] = currency_encoder.fit_transform(simulation_data.Currency)

In [27]:
class PandasData(bt.feeds.PandasData):
    lines = (
        'macd',
        'rsi',
        'bb_high',
        'bb_mid',
        'bb_low',
        'atr',
        'natr',
        'currency_volume',
        'adj_close',
        'month',
        'weekday',
        'ticker',
        'currency'
    )
    params = (
        ('datetime', None),
        ('open', 0),
        ('high', 1),
        ('low', 2),
        ('close', 3),
        ('volume', 4),
        ('macd', 5),
        ('rsi', 6),
        ('bb_high', 7),
        ('bb_mid', 8),
        ('bb_low', 9),
        ('atr', 10),
        ('natr', 11),
        ('currency_volume', 12),
        ('adj_close', 13),
        ('month', 14),
        ('weekday', 15),
        ('ticker', 16),
        ('currency', 17)
    )

In [28]:
test = []

In [46]:
simulation_data.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'MACD', 'RSI', 'BB_High',
       'BB_Mid', 'BB_Low', 'ATR', 'NATR', 'Currency_Volume', 'Adj Close',
       'Month', 'Weekday', 'Ticker', 'Currency', 'Ticker_Name'],
      dtype='object')

In [65]:
import numpy as np

class TestStrategy(bt.Strategy):
    def __init__(self): pass
    def next(self):
        market = self.get_market()
        print(market[market.Ticker == 'PBI'][['Close', 'Date']])
    # Organize Market Data in a DataFrame
    def get_market(self):
        market_data = pd.DataFrame()
        for i, d in enumerate(self.datas):
            # Fecha, Nombre del ticker
            dt, dn = self.datetime.date(), d._name
            # Posicion en el broker
            pos = self.getposition(d).size
            ticker_data = pd.DataFrame({
                'Date': [dt], 'Position': [pos], 'Open': [d.open[0]], 'High': [d.high[0]],
                'Low': [d.low[0]], 'Close': [d.close[0]], 'Volume': [d.volume[0]],
                'MACD': [d.macd[0]], 'RSI': [d.rsi[0]], 'BB_High': [d.bb_high[0]],
                'BB_Mid': [d.bb_mid[0]], 'BB_Low': [d.bb_low[0]], 'ATR': [d.atr[0]],
                'NATR': [d.natr[0]], 'Currency_Volume': [d.currency_volume[0]],
                'Adj Close': [d.adj_close[0]], 'Month': [d.month[0]],
                'Weekday': [d.weekday[0]], 'Ticker': [dn], 'Currency': [d.currency[0]]
            })
            market_data = market_data.append(ticker_data)
        return market_data

In [66]:
cerebro = bt.Cerebro()
cerebro.broker.setcash(100000.0)


for ticker in simulation_data.Ticker_Name.unique():
    ticker_mask = simulation_data.Ticker_Name == ticker
    ticker_data = simulation_data.loc[ticker_mask]
    cerebro.adddata(PandasData(dataname=ticker_data), name = ticker)

cerebro.addstrategy(TestStrategy)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
   Close        Date
0   8.68  2018-07-25
   Close        Date
0   8.84  2018-07-26
Final Portfolio Value: 100000.00


In [67]:
simulation_data[['Ticker_Name', 'Close']][simulation_data.Ticker_Name == 'PBI']

,Ticker_Name,Close
Date,,
2018-07-25,PBI,8.68
2018-07-26,PBI,8.84
